# SI 330: Homework 8: Databases & SQL


## Due: Friday, March 23, 2018,  11:59:00pm

### Submission instructions</font>
After completing this homework, you should turn in two files via Canvas ->  Assignments -> HW 7:
Your Notebook, named si330-hw7-YOUR_UNIQUE_NAME.ipynb and
the HTML file, named si330-hw7-YOUR_UNIQUE_NAME.html.

### Name:  YOUR NAME GOES HERE
### Uniqname: YOUR UNIQNAME GOES HERE
### People you worked with: [if you didn't work with anyone else write "I worked by myself" here].


## Learning Objectives
After completing this Homework, you should know how to:
* create a postgreSQL database on AWS Relational Database Service (RDS)
* load the contents of a CSV file into a pandas DataFrame
* manipulate a pandas DataFrame to extract information from a column
* load the resulting dataframes into a postgreSQL database
* issue SQL queries to answer real-world questions

## Background
We have generated a CSV file that consists of nearly 10,000 books from Goodreads.com.  
The file is derived from a dataset called "goodreads-10k" (see http://fastml.com/goodbooks-10k-a-new-dataset-for-book-recommendations/ for more details; we are only using the books.csv file for this assignment).

The purpose of this assignment is to take that CSV file, manipulate some of the data, and
write the resulting dataset to a postgreSQL database. Whereas this may seem contrived,
it models a real-world scenario in which the data would the be made available via
an API (which may be a nice bonus assignment, don't you think?).

We have identified six (6) steps beyond the initial setup to guide you through this 
assignment.  Each of the following steps is detailed below:
* Step 0: Redefine a function to improve performance and load the required libraries
* Step 1: Fill in the correct  parameters to create a new AWS RDS instance
* Step 2. Read the CSV file
* Step 3. Make a connection to your SQL Database
* Step 4: Breaking out the "authors"
* Step 5: Load your authors table into your postgreSQL database
* Step 6. Query the database

Places where you need to do something are indicated in <font color="magenta">magenta</font>.

### Step 0: Redefine a function to improve performance and load the required libraries

In [19]:
# This must be run before import pandas
# See pandas issue #8953 for an explanation
from pandas.io.sql import SQLTable

def _execute_insert(self, conn, keys, data_iter):
    print("Using monkey-patched _execute_insert")
    data = [dict((k, v) for k, v in zip(keys, row)) for row in data_iter]
    conn.execute(self.insert_statement().values(data))

SQLTable._execute_insert = _execute_insert

# Load required libraries
import pandas as pd
from sqlalchemy import create_engine

### Step 1: Fill in the correct  parameters to create a new AWS RDS instance
You should review Lab 8, log into your AWS Management Console, go to the RDS panel and 
create a new instance called si330wn2018-goodbooks-UNIQNAME (where UNIQNAME is replaced with your uniqname).

Create a database called "goodbooks", with your choice of username and password.

After your database is created, 
** <font color="magenta">record the relevant parameters in the following code block: **


In [20]:
endpoint = "si330wn2018-goodbooks-libbymur.clqth2h5kxt9.us-east-2.rds.amazonaws.com"
username = "root"
password = "libbymurray"

In [21]:
# DO NOT MODIFY THIS
database = "goodbooks" # we asked you to use this name for the database

### Step 2. Read the CSV file

We have provided you the data as a csv file on a public S3 bucket as https://s3.amazonaws.com/si330w18-cteplovs/books.csv. 
The file has four columns, of which you'll only deal with `authors` and `book_id` for this assignment.
You'll need to read the csv file (remember that you can pass a URL to ```pd.read_csv()```)and create a dataframe from it.
<font color="magenta">** Read the csv file and create the DataFrame. The datafile has 9980 rows in it; you should confirm that your dataframe has the correct number of rows. **</font>

In [22]:
books = pd.read_csv('https://s3.amazonaws.com/si330w18-cteplovs/books.csv')
print(len(books))

row_count=(len(books.index))
print(row_count)


books.head()

9980
9980


,id,authors,original_publication_year,title,language_code
0,1,Suzanne Collins,2008,"The Hunger Games (The Hunger Games, #1)",eng
1,2,"J.K. Rowling, Mary GrandPré",1997,Harry Potter and the Sorcerer's Stone (Harry P...,eng
2,3,Stephenie Meyer,2005,"Twilight (Twilight, #1)",en-US
3,4,Harper Lee,1960,To Kill a Mockingbird,eng
4,5,F. Scott Fitzgerald,1925,The Great Gatsby,eng


### Step 3. Make a connection to your SQL Database

In this step we will be using the SQLAlchemy package to connect to and query the database.
SQLAlchemy has many uses.  We're using it to allow us to use the pandas to_sql() function. 
We've provided the code to make the connection assuming that you have updated the variable in Step 1 above.

In [23]:
# This will establish a connection to your database, if you've set everything up properly
engine = create_engine('postgresql://{0}:{1}@{2}:5432/{3}'.format(username,password,endpoint,database))

In the next code block, we are using the pandas method ```.to_sql()``` to write a table to the database. ```.to_sql()``` takes in 3 parameters: the table to be used, the SQLAlchemy connection to the database, and what to do if the table already exists.

In [24]:
books.to_sql('books',engine,if_exists='replace')

OperationalError: (psycopg2.OperationalError) could not connect to server: Operation timed out
	Is the server running on host "si330wn2018-goodbooks-libbymur.clqth2h5kxt9.us-east-2.rds.amazonaws.com" (18.188.140.120) and accepting
	TCP/IP connections on port 5432?


### Step 4: Breaking out the "authors"
If you take a look at the books dataframe, you'll see the format of the authors field:
```
3160       James Patterson, Michael Ledwidge
1278               Virgil, Robert Fitzgerald
3106                       Patricia Cornwell
4764                          Scott B. Smith
1207                         Jennifer Probst
9703                        Orson Scott Card
7292                              R.L. Stine
8612                             Bella Andre
8718                             John Irving
7167    Arnaldur Indriðason, Bernard Scudder
```
The author column contains all the authors of a book separated by commas.
While this is useful to a human reader, it makes it very inefficient to 
search for a particular author, especially if the author you're looking
for isn't the first author of the book.

For that reason, we need to extract the author information and
create a new DataFrame called authors.  The authors dataframe should consist of two columns: "author" and "bookid" (use lowercase for the column names -- just trust us on this one).  The dataframe should look like:
```
     author          bookid
0	Suzanne Collins 1
1	J.K. Rowling    2
2	Mary GrandPré   2
3	Stephenie Meyer 3
4	Harper Lee      4
```

<font color="magenta">** Create a pandas DataFrame called authors, based on the contents of the books table.**</font>
Note that you will need to split the authors column on commas and strip whitespace from the resulting strings.

In [ ]:
authors = books[['authors','id']].copy()
authors.columns=['author','bookid']



print (type(authors))


### Step 5: Load your authors table into your postgreSQL database
Assuming you've created a pandas DataFrame called "authors", 
<font color="magenta">** you should be able to 
just run the following code block:**</font>

In [ ]:
authors.to_sql('authors',engine,if_exists='replace')

### Step 6. Query the database

Now that you've set everything up, you should answer the following questions **USING SQL and based on the data provided**:

Note: It might be better if you look at some reference on how to make SQL queries with SQLAlchemy. Conceptually and syntactically, it's similar to how you made queries with postgres.

<font color="magenta">**1. Confirm that the number of unique (Hint: DISTINCT) authors is 5829.**</font>

In [ ]:
# number of unique (Hint: DISTINCT) authors
inspector = inspect(authors)

<font color="magenta">**2.How many books has J.K. Rowling written?**</font>

In [31]:
# your code goes here

<font color="magenta">**3. Who has co-authored books with J.K. Rowling?**</font>

In [32]:
# your code goes here

<font color="magenta">**4. Which book has the largest number of authors?  How many authors?  Who are they?**</font>

In [30]:
# your code goes here

### Above and Beyond

Besides the `books.csv` we have also provided two other csv files. You can use them to upload to create tables to your RDS and use it for this section. Make sure that you have used SQL in your above and beyond.

Please also indicate in a separate markdown block why you think your work goes above and beyond.